In [1]:
import os

import itertools
import pickle
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math 

import sys
sys.path.append('../')
import utils
import wiki_utils
%matplotlib inline

# ========

In [2]:
import torchtext
from torchtext import data
from torchtext.datasets import WikiText2
# import spacy

tokenizer = lambda x: list(x)

TEXT = data.Field(lower=True, tokenize=tokenizer)

In [3]:
train, valid, test = WikiText2.splits(TEXT)

In [4]:
TEXT.build_vocab(train)

In [5]:
len(TEXT.vocab.stoi)

245

In [6]:
batch_size = 128
sequence_length = 30
grad_clip = 0.1
lr = 4.
best_val_loss = None
log_interval = 100
eval_batch_size = 128

In [7]:
train_iter, valid_iter, test_iter = data.BPTTIterator.splits(
    (train, valid, test),
    batch_size=batch_size,
    bptt_len=sequence_length, # this is where we specify the sequence length
    device=0,
    repeat=False)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [17]:
class CustomLoader:
    def __init__(self, data_iter):
        self.data_iter = data_iter
        self.iter = iter(data_iter)
        
    def __iter__(self):
        for d in self.data_iter:
#             d = next(self.data_iter)
            yield d.text, d.target.view(-1)
        
    def __len__(self):
        return int(len(train_iter.dataset.examples[0].text)/128)

In [9]:
train_loader = CustomLoader(train_iter)
val_loader = CustomLoader(valid_iter)
test_loader = CustomLoader(test_iter)

In [71]:
b = next(iter(train_iter)); vars(b).keys()

dict_keys(['batch_size', 'dataset', 'fields', 'text', 'target'])

In [69]:
for batch, (data, targets) in enumerate(train_loader):
    print(len(train_loader))
    break

128


In [59]:
data.size()

torch.Size([30, 128])

In [60]:
targets.size()

torch.Size([3840])

In [38]:

int(len(train.examples[0].text)/128/30)

AttributeError: 'function' object has no attribute 'examples'

In [16]:
len(train_loader)

2807

In [61]:
# corpus = wiki_utils.Texts('./wikitext/')

In [62]:
# eval_batch_size = 128
# train_loader = wiki_utils.TextLoader(corpus.train, batch_size=batch_size)
# val_loader = wiki_utils.TextLoader(corpus.valid, batch_size=eval_batch_size)
# test_loader = wiki_utils.TextLoader(corpus.test, batch_size=eval_batch_size)

In [10]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, hidden=None):
        emb = self.drop(self.encoder(x))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (weight.new(self.nlayers, bsz, self.nhid).zero_(),
                    weight.new(self.nlayers, bsz, self.nhid).zero_())
        else:
            return weight.new(self.nlayers, bsz, self.nhid).zero_()

In [11]:
def evaluate(data_loader):
    model.eval()
    total_loss = 0
    ntokens = len(TEXT.vocab.stoi) # len(corpus.dictionary)
    hidden = model.init_hidden(eval_batch_size)
    for i, (data, targets) in enumerate(data_loader):
        output, hidden = model(data)
        output_flat = output.view(-1, ntokens)
        total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / len(data_loader)

In [12]:
def train():
    model.train()
    total_loss = 0
    ntokens = len(TEXT.vocab.stoi) # len(corpus.dictionary)
    for batch, (data, targets) in enumerate(train_loader):
        model.zero_grad()
        output, hidden = model(data)
#         print(targets.size(), output.view(-1, ntokens).size())
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_loader) // sequence_length, lr, cur_loss, math.exp(cur_loss)))
            total_loss = 0

In [13]:
ntokens = len(TEXT.vocab.stoi) # len(corpus.dictionary)
model = RNNModel('LSTM', ntokens, 128, 128, 2, 0.3)
criterion = nn.CrossEntropyLoss()

In [14]:
def generate(n=50, temp=1.):
    model.eval()
    x = torch.rand(1, 1).mul(ntokens).long()
    hidden = None
    out = []
    for i in range(n):
        output, hidden = model(x, hidden)
        s_weights = output.squeeze().data.div(temp).exp()
        s_idx = torch.multinomial(s_weights, 1)[0]
        x.data.fill_(s_idx)
        # s = corpus.dictionary.idx2symbol[s_idx]
        s = TEXT.vocab.itos[s_idx]
        out.append(s)
    return ''.join(out)

In [15]:
with torch.no_grad():
    print('sample:\n', generate(50), '\n')

for epoch in range(1, 6):
    train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')


sample:
 ơ€มав̃šh;кấ礮′‑礮ჯწq·6eი'kცṯễ!ต⅔ōòlşč−殻0š±a&ხúø%é空¡大 

| epoch   1 |   100/   93 batches | lr 4.00 | loss  3.46 | ppl    31.98
| epoch   1 |   200/   93 batches | lr 4.00 | loss  3.15 | ppl    23.45
| epoch   1 |   300/   93 batches | lr 4.00 | loss  3.13 | ppl    22.78


KeyboardInterrupt: 

In [11]:
t1 = generate(10000, 1.)
t15 = generate(10000, 1.5)
t075 = generate(10000, 0.75)
with open('./generated075.txt', 'w') as outf:
    outf.write(t075)
with open('./generated1.txt', 'w') as outf:
    outf.write(t1)
with open('./generated15.txt', 'w') as outf:
    outf.write(t15)